<a href="https://colab.research.google.com/github/kibrus/Patient-Survival-Predictor/blob/main/patient_Survival_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import math

# Data manipulation
import numpy as np
import pandas as pd

# plotting and visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
sns.set_theme()
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [ ]:
#Train-test split and K-fold cross validation
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

In [ ]:
# missing data imputtation
from sklearn.impute import SimpleImputer

#categorical data encoding
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Deep learning framework
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
#hyperparameter Tuning
!pip install -q -U keras-tuner
import keras_tuner as kt
from keras_tuner import HyperModel, Hyperband

In [ ]:
#model Evaluation
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
#model loading
from keras.models import load_model
#Explainable AI
!pip install shap
import shap

#warning suppression
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#recording the starting time
start = time.time()

# Data

In [ ]:
#loading the data
data = pd.read_csv('/content/Dataset.csv')

#print the datafram
data

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.730000,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.420000,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.950000,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.640000,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,91592,78108,30,0,75.0,23.060250,0,Caucasian,M,177.8,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,Sepsis,Cardiovascular
91709,66119,13486,121,0,56.0,47.179671,0,Caucasian,F,183.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
91710,8981,58179,195,0,48.0,27.236914,0,Caucasian,M,170.2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
91711,33776,120598,66,0,NaN,23.297481,0,Caucasian,F,154.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory


In [ ]:
# Memory usage
print("Memory usage: {0:.2f} MB".format(data.memory_usage().sum()/(1024*1024)))

Memory usage: 130.15 MB


# Understanding Features using Data Dictionary

In [ ]:
data_dictionary = pd.read_csv('/content/Data Dictionary.csv')

#printing the dataframe
data_dictionary

,Category,Variable Name,Unit of Measure,Data Type,Description,Example
0,identifier,encounter_id,None,integer,Unique identifier associated with a patient un...,None
1,identifier,hospital_id,None,integer,Unique identifier associated with a hospital,None
2,identifier,patient_id,None,integer,Unique identifier associated with a patient,None
3,demographic,hospital_death,None,binary,Whether the patient died during this hospitali...,0
4,demographic,age,Years,numeric,The age of the patient on unit admission,None
...,...,...,...,...,...,...
183,APACHE comorbidity,lymphoma,None,binary,Whether the patient has been diagnosed with no...,1
184,APACHE comorbidity,solid_tumor_with_metastasis,None,binary,Whether the patient has been diagnosed with an...,1
185,APACHE grouping,apache_3j_bodysystem,None,string,Admission diagnosis group for APACHE III,Cardiovascular
186,APACHE grouping,apache_2_bodysystem,None,string,Admission diagnosis group for APACHE II,Respiratory


In [ ]:
print("Memory usage: {0:.2f} MB".format(data_dictionary.memory_usage().sum()/(1024*1024)))

Memory usage: 0.01 MB


In [ ]:
#Shape of the data
print(data.shape)

(91713, 186)


In [ ]:
#column datatypes
print(data.dtypes)

encounter_id                     int64
patient_id                       int64
hospital_id                      int64
hospital_death                   int64
age                            float64
                                ...   
leukemia                       float64
lymphoma                       float64
solid_tumor_with_metastasis    float64
apache_3j_bodysystem            object
apache_2_bodysystem             object
Length: 186, dtype: object


In [ ]:
# count column datatypes
data.dtypes.value_counts()

float64    170
int64        8
object       8
dtype: int64

In [ ]:
# column datatypes
cols_int = data.columns[data.dtypes == 'int64'].tolist()
cols_float = data.columns[data.dtypes == 'float64'].tolist()
cols_object = data.columns[data.dtypes == 'object'].tolist()

In [ ]:
# number of duplicated rows in our data
print(f'Number of duplicate rows in the dataset:  {data.duplicated().sum()}')

Number of duplicate rows in the dataset:  0


In [ ]:
# number of duplicated columns in our data
print(f'Number of duplicate columns in the dataset:  {data.T.duplicated().sum()}')

Number of duplicate columns in the dataset:  3


In [ ]:
#count of columns with missing values
print(f'Number of columns with with missing values: {len(data.isna().sum()[data.isna().sum() != 0])} out of {len(data.columns)}')

Number of columns with with missing values: 175 out of 186


In [ ]:
#get statistical desctiption of the dataset
data.describe()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,...,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,91713.000000,91713.000000,91713.000000,91713.000000,87485.000000,88284.000000,91713.000000,90379.000000,91713.000000,91713.000000,...,83766.000000,83766.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000,90998.000000
mean,65606.079280,65537.131464,105.669262,0.086302,62.309516,29.185818,0.183736,169.641588,508.357692,0.835766,...,0.086787,0.043955,0.000857,0.015693,0.225192,0.012989,0.026165,0.007066,0.004132,0.020638
std,37795.088538,37811.252183,62.854406,0.280811,16.775119,8.275142,0.387271,10.795378,228.989661,2.487756,...,0.247569,0.217341,0.029265,0.124284,0.417711,0.113229,0.159628,0.083763,0.064148,0.142169
min,1.000000,1.000000,2.000000,0.000000,16.000000,14.844926,0.000000,137.200000,82.000000,-24.947222,...,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32852.000000,32830.000000,47.000000,0.000000,52.000000,23.641975,0.000000,162.500000,369.000000,0.035417,...,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,65665.000000,65413.000000,109.000000,0.000000,65.000000,27.654655,0.000000,170.100000,504.000000,0.138889,...,0.050000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,98342.000000,98298.000000,161.000000,0.000000,75.000000,32.930206,0.000000,177.800000,679.000000,0.409028,...,0.130000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,131051.000000,131051.000000,204.000000,1.000000,89.000000,67.814990,1.000000,195.590000,927.000000,159.090972,...,0.990000,0.970000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# Droping constant columns
data.drop(columns=data.columns[data.nunique() == 1], inplace=True)

In [ ]:
data.shape

(91713, 185)

# Target variable
The target variable hospital_death is a binary variable indicating the survival status of a patient.
* 0 -> patient survived
* 1 -> patient died


In [ ]:
# Function to construct barplot and donutplot of a dataframe column
def bar_donut(df, col, height = 500, width = 800, manual_title_text = False, title_text = "Frequency distribution"):
    fig = make_subplots(rows = 1, cols = 2, specs = [[{'type': 'xy'}, {'type': 'domain'}]])
    fig.add_trace(go.Bar(x = df[col].value_counts(sort = False).index.tolist(),
                         y = df[col].value_counts(sort = False).tolist(),
                         text = df[col].value_counts(sort = False).tolist(),
                         textposition = 'auto'),
                         row = 1, col = 1)
    fig.add_trace(go.Pie(values = df[col].value_counts(sort = False).tolist(),
                         labels = df[col].value_counts(sort = False).index.tolist(),
                         hole = 0.5, textinfo = 'label+percent', title = f"{col}"),
                         row = 1, col = 2)
    fig.update_layout(height = height, width = width, showlegend = False,
                      title = {'text': f"Frequency distribution of {col}",
                               'y': 0.95, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'},
                      xaxis = dict(tickmode = 'linear', tick0 = 0, dtick = 1))
    if manual_title_text == True:
        fig.update_layout(height = height, width = width, showlegend = False,
                          title = {'text': title_text,
                                   'y': 0.95, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'},
                          xaxis = dict(tickmode = 'linear', tick0 = 0, dtick = 1))
    fig.show()

In [ ]:

# hospital_death
bar_donut(data, 'hospital_death')

The dataset is imbalanced with respect to the target variable

In [ ]:
# Donutplots in N x 3 grid form
def donuts_grid(df, cols, ncols = 3, hole = 0.5, height = 1500, width = 900):
    nrows = math.ceil(len(cols)/ncols)
    specs = np.full((nrows, ncols), {'type': 'domain'}).tolist()
    fig = make_subplots(rows = nrows, cols = ncols, specs = specs)
    count = 0
    break_flag = False
    for row in range(nrows):
        for col in range(ncols):
            i = (row * ncols) + col
            fig.add_trace(go.Pie(values = df[cols[i]].value_counts(sort = False).tolist(),
                                 labels = df[cols[i]].value_counts(sort = False).index.tolist(),
                                 hole = hole, textinfo = 'percent', title = f'{cols[i]}'),    # 'label+percent'
                          row = row + 1, col = col + 1)
            count = count + 1
            if count == len(cols):
                break_flag = True
                break

        if break_flag == True:
            break

    fig.update_layout(height = height, width = width)
    fig.show()

In [ ]:
# Binary columns except gender and hospital_death
cols_binary = [col for col in data.columns if data[col].nunique() == 2]
cols_binary.remove('gender')
cols_binary.remove('hospital_death')
donuts_grid(data, cols_binary, ncols = 4, hole = 0.5, height = 1250, width = 1200)

In [ ]:
# Number of unique values
keys = ['encounter_id', 'patient_id', 'hospital_id', 'icu_id']
data[keys].nunique()

encounter_id    91713
patient_id      91713
hospital_id       147
icu_id            241
dtype: int64

Observation: The features encounter_id and patient_id are unique for each observation, and hence do not contribute to the task of predicting the target variable.

In [ ]:
# Dropping encounter_id and patient_id
data.drop(['encounter_id', 'patient_id'], axis = 1, inplace = True)

# Data Preprocessing

In [ ]:
# train-test
X = data.drop('hospital_death', axis=1) #features
Y = data['hospital_death'] #target

### Missing Data Imputation

In [ ]:
#count missing values for the target variable
print(f'Number of missing trget values: {Y.isnull().sum()}')

Number of missing trget values: 0


In [ ]:
# Columns with more than 50% missing values in the training set
print((X.isna().sum()/len(X))[X.isna().sum()/len(X) > 0.5])

albumin_apache          0.592926
bilirubin_apache        0.633869
fio2_apache             0.772715
paco2_apache            0.772715
paco2_for_ph_apache     0.772715
                          ...   
h1_arterial_ph_min      0.833295
h1_arterial_po2_max     0.828072
h1_arterial_po2_min     0.828072
h1_pao2fio2ratio_max    0.874413
h1_pao2fio2ratio_min    0.874413
Length: 74, dtype: float64


In [ ]:
missing_more = (X.isnull().sum()/len(X))[X.isnull().sum()/len(X)>0.5].index.tolist()
X.drop(missing_more, axis = 1, inplace= True)

# Proportion-based imputation
With the goal of keeping the feature distributions same before and after imputation, we impute the missing values in a column in such a way so that the proportions of the existing unique values in that particular column remain roughly same as those were prior to the imputation. The following function takes a dataframe, implements the proportion-based imputation in each column containing missing values and returns the resulting dataframe.

In [ ]:
# Function to impute missing values proportionately with respect to the existing unique values
def prop_imputer(df):
    df_prop = df.copy(deep = True)
    missing_cols = df_prop.isna().sum()[df_prop.isna().sum() != 0].index.tolist()
    for col in missing_cols:
        values_col = df_prop[col].value_counts(normalize = True).index.tolist()
        probabilities_col = df_prop[col].value_counts(normalize = True).values.tolist()
        df_prop[col] = df_prop[col].fillna(pd.Series(data = np.random.choice(values_col, p = probabilities_col, size = len(df_prop)), index = df_prop[col].index))
    return df_prop

In [ ]:
# Proportion-based imputation
X = prop_imputer(X)

In [ ]:
#chack if there is any missing value
print(len((X.isnull().sum())[X.isnull().sum()/len(X)>0]))

0


In [ ]:
# Label encoding
def label_encoder(df, cols):
    df_le = df.copy(deep = True)
    le = LabelEncoder()
    for col in cols:
        df_le[col] = le.fit_transform(df_le[col])
    return df_le

In [ ]:
X_le = label_encoder(X, cols_object) # numerical values between 0 and n-1.

In [ ]:
X_le

,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,118,68.0,22.730000,0,2,1,180.3,4,1,92,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9,0
1,81,77.0,27.420000,0,2,0,160.0,4,1,90,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8,6
2,118,25.0,31.950000,0,2,0,172.7,3,0,93,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,3
3,118,81.0,22.640000,1,2,0,165.1,8,2,92,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,33,19.0,32.105085,0,2,1,188.0,3,0,91,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,30,75.0,23.060250,0,2,1,177.8,0,1,927,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,9,0
91709,121,56.0,47.179671,0,2,0,183.0,3,1,925,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0
91710,195,48.0,27.236914,0,2,1,170.2,3,0,908,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,3
91711,66,84.0,23.297481,0,2,0,154.9,3,0,922,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,6


In [ ]:
# Function for one-hot encoding
def one_hot_encoder(df, cols, drop_first = False):
    cols = [col for col in cols if col in df.columns] # To ensure that 'cols' is contained (as a subset) within df.columns
    df_ohe = pd.get_dummies(df, columns = cols, drop_first = drop_first)
    return df_ohe

Note that gender_F and gender_M are related by gender_F + gender_M = 1. Hence we shall lose no information by dropping one of these columns. This can be done (for each feature) by changing drop_first = False to drop_first = True.

In [ ]:
# One-hot encoding with drop_first = True
X_ohe = one_hot_encoder(X, cols_object, drop_first = True)
X=X_ohe

# Normalization

In [ ]:
# Min-max normalization of predictors in the training set
for col in X.columns:
    if X[col].dtypes == 'int64' or X[col].dtypes == 'float64': # Checking if the column is numerical
        if X[col].nunique() > 1: # Checking if the column is non-constant
            X[col] = (X[col] - X[col].min()) / (X[col].max() - X[col].min())
X

,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,...,apache_3j_bodysystem_Trauma,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,0.574257,0.712329,0.148859,0.0,0.738140,0.011834,0.138498,0.239484,0.057971,0.228074,...,0,0,0,0,0,0,0,0,0,0
1,0.391089,0.835616,0.237400,0.0,0.390478,0.009467,0.140596,0.214383,0.033816,0.092229,...,0,0,0,0,0,0,1,0,0,0
2,0.574257,0.123288,0.322920,0.0,0.607981,0.013018,0.135558,0.384668,0.101449,0.319404,...,0,0,0,1,0,0,0,0,0,0
3,0.574257,0.890411,0.147160,1.0,0.477822,0.011834,0.135558,0.156716,0.492754,0.547932,...,0,0,0,0,0,0,0,0,0,0
4,0.153465,0.041096,0.325847,0.0,0.870012,0.010651,0.135955,0.291045,0.086957,0.273053,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,0.138614,0.808219,0.155094,0.0,0.695325,1.000000,0.137177,0.232700,0.057971,0.227642,...,0,0,0,0,0,0,0,0,0,0
91709,0.589109,0.547945,0.610434,0.0,0.784381,0.997633,0.136207,0.810041,0.057971,0.227638,...,0,0,0,0,0,0,0,0,0,0
91710,0.955446,0.438356,0.233943,0.0,0.565165,0.977515,0.135807,0.273406,0.106280,0.318940,...,0,0,0,1,0,0,0,0,0,0
91711,0.316832,0.931507,0.159572,0.0,0.303134,0.994083,0.136000,0.117368,0.033816,0.092229,...,0,0,0,0,0,0,1,0,0,0


# train-test-cv split

In [ ]:
# train-test-cv split
X_train, X_, Y_train, Y_ = train_test_split(X,Y, test_size = 0.4, shuffle=True, random_state=24, stratify=Y)
X_cv, X_test, Y_cv, Y_test = train_test_split(X_,Y_, test_size=0.5, shuffle= True, random_state=24, stratify=Y_)

In [ ]:
X_cv.shape, Y_cv.shape, X_test.shape, Y_test.shape,X.shape, Y_train.shape

((18343, 152), (18343,), (18343, 152), (18343,), (91713, 152), (55027,))

# Baseline Neural Network

In [ ]:
model_1 = Sequential([
    Dense(16, input_dim = len(X_train.columns), activation = 'relu'),
    Dense(12, activation = 'relu'),
    Dense(8, activation = 'relu'),
    Dense(4, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 16)                2448      
                                                                 
 dense_6 (Dense)             (None, 12)                204       
                                                                 
 dense_7 (Dense)             (None, 8)                 104       
                                                                 
 dense_8 (Dense)             (None, 4)                 36        
                                                                 
 dense_9 (Dense)             (None, 1)                 5         
                                                                 
Total params: 2797 (10.93 KB)
Trainable params: 2797 (10.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_1.compile(loss='binary_crossentropy',
              optimizer = keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history_1 = model_1.fit(X_train, Y_train,
                        validation_data=(X_cv,Y_cv),
                        epochs=100,
                        batch_size=64)

Epoch 1/100
860/860 [==============================] - 4s 3ms/step - loss: 0.2378 - accuracy: 0.9164 - val_loss: 0.2102 - val_accuracy: 0.9234
Epoch 2/100
860/860 [==============================] - 3s 3ms/step - loss: 0.2114 - accuracy: 0.9227 - val_loss: 0.2059 - val_accuracy: 0.9244
Epoch 3/100
860/860 [==============================] - 3s 4ms/step - loss: 0.2081 - accuracy: 0.9241 - val_loss: 0.2032 - val_accuracy: 0.9245
Epoch 4/100
860/860 [==============================] - 2s 3ms/step - loss: 0.2067 - accuracy: 0.9242 - val_loss: 0.2021 - val_accuracy: 0.9254
Epoch 5/100
860/860 [==============================] - 3s 3ms/step - loss: 0.2049 - accuracy: 0.9252 - val_loss: 0.2034 - val_accuracy: 0.9249
Epoch 6/100
860/860 [==============================] - 3s 3ms/step - loss: 0.2041 - accuracy: 0.9252 - val_loss: 0.2084 - val_accuracy: 0.9235
Epoch 7/100
860/860 [==============================] - 5s 6ms/step - loss: 0.2027 - accuracy: 0.9259 - val_loss: 0.2019 - val_accuracy: 0.9248